In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
df = pd.read_csv('out/csv/incorrect/1.csv')
# df = pd.concat([df, pd.read_csv('out/csv/correct/1.csv')])
df = pd.concat([df, pd.read_csv('out/csv/incorrect/2.csv'),
                pd.read_csv('out/csv/correct/1.csv'),
                pd.read_csv('out/csv/correct/2.csv')])

In [2]:
df = None

In [3]:
for root, dir, files in os.walk("out/csv/incorrect"):
    for file in files:
        if file.endswith(".csv"):
            df = pd.concat([df, pd.read_csv(os.path.join(root, file))])

In [4]:
for root, dir, files in os.walk("out/csv/correct"):
    for file in files:
        if file.endswith(".csv"):
            df = pd.concat([df, pd.read_csv(os.path.join(root, file))])

In [6]:
x = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [7]:
x.shape

(4172, 12)

In [27]:
# x = x.reshape(len(x), 1, x.shape[1])

In [8]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [9]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x, y, train_size=0.8, test_size=0.2, random_state=1)

In [18]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, Reshape
from keras.optimizers import Adam

model = Sequential([
    Reshape((1, x.shape[1]), input_shape=(x.shape[1],)), # 12 is the number of features
    GRU(128, activation='relu', return_sequences=True),
    GRU(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

opt_adam = Adam(learning_rate=1e-3)

model.compile(optimizer=opt_adam, loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
model.fit(x_train, y_train, epochs=200, validation_data=(x_test, y_test))

Epoch 1/200
105/105 [==============================] - 1s 8ms/step - loss: 0.5054 - accuracy: 0.7297 - val_loss: 0.4986 - val_accuracy: 0.7377
Epoch 2/200
105/105 [==============================] - 1s 6ms/step - loss: 0.4965 - accuracy: 0.7387 - val_loss: 0.4966 - val_accuracy: 0.7413
Epoch 3/200
105/105 [==============================] - 0s 5ms/step - loss: 0.4955 - accuracy: 0.7390 - val_loss: 0.4993 - val_accuracy: 0.7317
Epoch 4/200
105/105 [==============================] - 1s 5ms/step - loss: 0.4987 - accuracy: 0.7348 - val_loss: 0.4935 - val_accuracy: 0.7425
Epoch 5/200
105/105 [==============================] - 1s 5ms/step - loss: 0.4985 - accuracy: 0.7339 - val_loss: 0.4899 - val_accuracy: 0.7329
Epoch 6/200
105/105 [==============================] - 1s 5ms/step - loss: 0.5010 - accuracy: 0.7288 - val_loss: 0.5005 - val_accuracy: 0.7461
Epoch 7/200
105/105 [==============================] - 0s 5ms/step - loss: 0.5005 - accuracy: 0.7384 - val_loss: 0.5070 - val_accuracy: 0.7377

In [21]:
model.save('out/models/model3.h5')

In [43]:
print(x[0].reshape(1, x.shape[1]))

[[2.63072924 2.46842228 1.93607542 2.44789053 2.77396979 3.23779275
  1.57094369 3.21220976 2.88081308 3.90075045 1.34429642 3.89212598]]


In [47]:
from keras.models import save_model

save_model(model, 'out/models/android.h5')

In [48]:
from tensorflow import lite
converter = lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
  lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tfmodel = converter.convert()
open('out/models/android.tflite', 'wb').write(tfmodel)

INFO:tensorflow:Assets written to: C:\Users\SIDDHA~1\AppData\Local\Temp\tmpnysh_pl0\assets


INFO:tensorflow:Assets written to: C:\Users\SIDDHA~1\AppData\Local\Temp\tmpnysh_pl0\assets


390348